# OpenABM-Covid19 - Example Digital Contact-Tracing Protection Calculation

This examples demonstrates the effect of digital contact-tracing following testing (same as Example Digital Contact-Tracing) and shows the protection gained by users of the system. Digital contract tracing is turned on following the end of a lockdown period and moderate social-distancing measures are added (20% reduction in transmission for occupation and random interactions).

First we set the baseline parameters: change the population to 200k (`n_total=200000`); the test sensitivity to 90% (`test_sensitivity=0.9`); and the fraction of app users to be 60% in the 10y-69y age groups (`app_useres_fraction_10_19=0.6` etc.) and 30% in the over 70s. At the start of the simulation there is no self-isolation and we run the simulation for 15 days.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
from COVID19.model import Model as abm

params = {
    "n_total" : 200000,
    "test_sensitivity" : 0.9,
    "app_users_fraction_0_9" : 0,
    "app_users_fraction_10_19" : 0.6,
    "app_users_fraction_20_29" : 0.6,
    "app_users_fraction_30_39" : 0.6,
    "app_users_fraction_40_49" : 0.6,
    "app_users_fraction_50_59" : 0.6,
    "app_users_fraction_60_69" : 0.6,
    "app_users_fraction_70_79" : 0.3,
    "app_users_fraction_80" : 0.3
}
model = abm( params = params)
for t in range(30):
    model.one_time_step()

After 30 days we implement a general lockdown (default parameters) and also introducing self-isolation of (80%) of individuals showing symptoms and their households.

In [2]:
model.update_running_params("lockdown_on", 1)
model.update_running_params("self_quarantine_fraction", 0.8 )
model.update_running_params("quarantine_household_on_symptoms", 1)
for t in range(30):
    model.one_time_step()

Following the lockdown we introduce digital contact tracing (`app_turned_on=1`). We use the set up that symptomatic individuals who are self-isolating get tests (`test_on_symptoms=1`), tracing occurs following a positive test (`trace_on_positive=1`) with those traced being asked to quarantine (`quarantine_on_traced=1`) and their household members also being asked to quarantined (`quarantine_household_on_traced_positive=1`). Social-distancing measures reduce the transmission rate by 20% in occupation and random interactions (`relative_transmission_occupation=0.8` etc.). We then the the simulation for another 50 days.

In [3]:
model.update_running_params("lockdown_on", 0)
model.update_running_params("app_turned_on", 1 )
model.update_running_params("test_on_symptoms", 1)
model.update_running_params("test_order_wait", 0 )
model.update_running_params("test_result_wait", 1 )
model.update_running_params("trace_on_positive", 1)
model.update_running_params("quarantine_on_traced", 1)
model.update_running_params("quarantine_household_on_traced_positive", 1 )
model.update_running_params("relative_transmission_occupation", 0.8 )
model.update_running_params("relative_transmission_random", 0.8)
for t in range(50):
    model.one_time_step()

Now we write all the transmissions from the model and all the individuals characteristic's to file and load to dataframes.

In [4]:
model.write_transmissions()
model.write_individual_file()
df_transmission = pd.read_csv( "transmission_Run1.csv", comment="#", sep=",", skipinitialspace=True )
df_indiv = pd.read_csv("individual_file_Run1.csv", comment="#", sep=",", skipinitialspace=True )

Finally we look at the number of infections (post DCT starting) by age group and for app users versus non app users and calculate the relative risk of infection. The simulation shows that app users are about 9% less likely to be infected than non-app users.

In [5]:
df_trans_post = df_transmission[ df_transmission[ "time_infected" ] > 60 ].loc[ :,{"ID_recipient"}]
df_trans_post[ "infected" ] = 1 
df = pd.merge( df_indiv, df_trans_post, left_on = "ID", right_on = "ID_recipient", how = "left")
df.fillna(0)
df[ "total" ] = 1
df = df.groupby( ["age_group", "app_user"] ).agg({ "infected": "sum", "total": "count"}).reset_index()
df[ "infected_fraction"] = df["infected"] / df["total"]
df = df.pivot_table( index =["age_group"], columns = "app_user", values = "infected_fraction").reset_index()
df[ "protection"] = 1 - df[1]/ df[0]
sys.stdout.write( "App user protection = %.1f%%" % ( 100 * df[ df["age_group"] > 0 ]["protection"].mean()) )
df.loc[ :,{"age_group", "protection"}]

App user protection = 8.7%

app_user,age_group,protection
0,0,NaN
1,1,0.082820
2,2,0.092547
3,3,0.043949
4,4,0.100965
5,5,0.031993
6,6,0.116402
7,7,0.181672
8,8,0.044974
